# Setup

## Imports and jupyter environment

In [ ]:
# Imports and basic environment setup
import plotly.offline as py
import plotly.express as px
from IPython.display import Markdown
import ipywidgets as widgets
import pandas as pd
import numpy as np
import pprint
import io
import os.path
import qgrid
import requests
import urllib.request


py.init_notebook_mode(connected=False)

# Use jupyter qgrid instead of standard table views
qgrid.enable(dataframe=True, series=True)
qgrid.set_grid_option('forceFitColumns', False)
qgrid.set_grid_option('editable', False)
qgrid.set_grid_option('minVisibleRows', 3)
qgrid.set_grid_option('maxVisibleRows', 15)

# Pandas
pd.options.display.max_rows = 999
pd.options.display.max_columns = 60

## Get covid data

In [ ]:
use_proxy = False

def proxy(url):
    if use_proxy:
        proxy_dict = {'https' : "https://10.144.1.10:8080"}
        s = requests.get(url, proxies=proxy_dict).text
        return(io.StringIO(s))
    else:
        return(url)
    

def get_n_melt_data(data_url, case_type):
    '''
        thanks to: 
        https://medium.com/@jcharistech/data-cleaning-a-practical-example-with-coronavirus-dataset-using-pandas-and-schedule-for-14abf485c881
    '''
    df = pd.read_csv(proxy(data_url))
    df = df.groupby('Country/Region', as_index=False).sum()
    melted_df = df.melt(id_vars=['Country/Region', 'Lat', 'Long'])
    melted_df.rename(columns={"variable":"Date","value":case_type, "Country/Region": "Country"},inplace=True)
    return melted_df

def merge_data(confirm_df, deaths_df):
    new_df = confirm_df.join(deaths_df['Deaths'])
    return new_df

# get the different CSVs; mapped around some columns/rows, and aggregate into a single df
confirmed_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
deaths_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
df_confirmed = get_n_melt_data(confirmed_url,'Confirmed')
df_deaths = get_n_melt_data(deaths_url,'Deaths') 

# merge the different data-frames into a single aggregated one
agg = merge_data(df_confirmed, df_deaths)
agg['Date'] = pd.to_datetime(agg['Date'])
agg = agg.sort_values(by=['Country', 'Date'])

# Dataset has gap on 3/12. Set 3/12 as avg between 3/11 and 3/13; not perfect but ok'ish
for case_type in ['Confirmed', 'Deaths']:
    d1 = agg[agg['Date'] == '2020-03-13'][case_type]
    d2 = agg[agg['Date'] == '2020-03-11'][case_type]
    d_result = d2.values+((d1.values-d2.values)/2)
    agg.loc[(agg.Date == '2020-03-12'),case_type]=d_result    

## Read population data

In [ ]:
def get_population_data():
    # Population from here: https://datahub.io/core/population/r/population.csv
    df_population = pd.read_csv('population.csv')
    # only use most recent year (=2016)
    df_population = df_population.sort_values(by=['Country Name', 'Year']).groupby('Country Name').tail(1)
    # we'll be merging this df soon, let's drop what we don't need
    df_population = df_population.drop(['Country Code', 'Year'], axis=1)
    # rename few columns
    df_population.rename(columns={"Value" : "Population", 'Country Name': 'Country'}, inplace=True)
    
    # Debug, just to get the exact country name for replace in below
    # for i, e in population.iterrows():
    #     print(e['Country'])

    # remap some country as naming is different between the two datasets (TODO: additional countries)    
    df_population['Country'] = df_population['Country'].replace(
                                                      {
                                                       'United States' : 'US',
                                                       'Iran, Islamic Rep.' : 'Iran',
                                                       "Korea, Dem. People’s Rep." : 'Korea, South',
                                                       'Russian Federation' : 'Russia',
                                                       'Egypt, Arab Rep.' : 'Egypt',
                                                       'Taiwan' : 'Taiwan*'
                                                      })
    return(df_population)


# get population data
population = get_population_data()    
agg = agg.merge(population, how='left')

## Calculate extra data

In [ ]:
# calculate a number of extra columns (delta, delta_pct, case_type per million etc)
for case_type in ['Confirmed', 'Deaths']:
    # Add daily increment per case_type
    col_name = case_type+'_Delta'
    agg[col_name] = agg.groupby(['Country'])[case_type].diff().fillna(0)
    # Add daily percentage increase per case_type
    col_name = case_type+'_Pct'
    agg[col_name] = agg.groupby(['Country'])[case_type].pct_change(periods=1)*100
    agg[col_name] = agg[col_name].round(1)
    # Add case_type per million population
    col_name = case_type+'_per_million'
    agg[col_name] = agg[case_type] / agg['Population'] * 1000000
    agg[col_name] = agg[col_name].round(1)
    
display(agg)
# figure out how long ago a country hit 50 infections as baseline when breakout started
df = agg[agg['Confirmed'] > 50].groupby('Country').first()
# Calculate datetime difference between now and first reported day with Confirmed > threshold
df['Days'] = pd.to_datetime("now") - df['Date']
# Convert to days
df['Days']= df['Days'].apply(lambda x: round(x / np.timedelta64(1,'D'),0))
# drop all columns except for index (=country) and 'Days'
df = df[['Days']]
agg = agg.merge(df['Days'], how='left', left_on='Country', right_index=True, suffixes=(False,False))

    
# Let's reorder the columns a bit; so that all case_type columns are together
agg.sort_index(axis=1, inplace=True)
# Sort will put them all together. Move around the other columns a bit
cols = agg.columns.tolist()
cols.insert(0, cols.pop(cols.index('Days')))
cols.insert(0, cols.pop(cols.index('Date')))
cols.insert(0, cols.pop(cols.index('Country')))
cols.insert(len(cols), cols.pop(cols.index('Population')))
cols.insert(len(cols), cols.pop(cols.index('Long')))
cols.insert(len(cols), cols.pop(cols.index('Lat')))
agg = agg.reindex(columns= cols)

## Setup all graphs

In [ ]:
import plotly.graph_objects as go
from datetime import date, timedelta

def print_totals_graph(df, category, title='',  rolling=1, log=True):
    fig = go.Figure()

    if title == '':
        title=category

    if rolling > 1:
        title = title + f" ({rolling} day rolling avg)"
        
    if log:
        fig.update_layout(yaxis_type="log")
        title = title + " (log)"
    
        
    fig.update_layout(title_text=title)
    
    df = df.groupby('Date')[category].sum()
    
    # let's calculate rolling avg
    with pd.option_context('mode.chained_assignment', None):
        df = df.rolling(window=rolling).mean()

    fig.add_trace(go.Scatter(
                            x=df.index,
                            y=df,
                            name=category))
    fig.show()
        
        
def print_graph(df, c_list, categories, title='', days=14, log=True, rolling=1, trendline=False):
    if c_list[0] == 'All':
        filtered = df.copy()
    else:
        filtered = df[df['Country'].isin(c_list)]
            
    end = date.today()
    start = end - timedelta(days=days)
    if title == '':
        title=categories[0]
         
    fig = go.Figure()
    
    if log:
        fig.update_layout(yaxis_type="log")
        title = title + " (log)"

    if rolling > 1:
        title = title + f" ({rolling} day rolling avg)"
        
    fig.update_layout(title_text=title, xaxis_range=[start.strftime("%Y-%m-%d"),end.strftime("%Y-%m-%d")])

    for country in filtered.Country.unique():
        df = filtered.loc[filtered['Country'] == country]
        for cat in categories:
            if len(categories) > 1:
                name=country+"_"+cat
            else:
                name=country
            
# TODO: Add trendline             
#             if trendline:
#                 tmpfig = px.scatter(df, x='Date', y=cat, color='Country', trendline="ols")
#                 #display(tmpfig)
#                 fig.add_trace(tmpfig.data[1], color=country)
# #                fig.add_trace(px.scatter_mapbox(stations, lat=“ycoord”, lon=“xcoord”,).data[0])

            # let's calculate rolling avg
            with pd.option_context('mode.chained_assignment', None):
                df[cat] = df[cat].rolling(window=rolling).mean()

            fig.add_trace(go.Scatter(
                            x=df['Date'],
                            y=df[cat],
                            name=name))
    fig.show()


def print_graph_from_start(df, c_list, categories, title='', days=14, rolling=1, log=True):
    ''' 
        graphs 'from_start' use Days column, which is the days-passed since a country 
        passed 50 infection threshold.
    '''
    if c_list[0] == 'All':
        filtered = df.copy()
    else:
        filtered = df[df['Country'].isin(c_list)]
       
    fig = go.Figure()
    
    if title == '':
        title=categories[0]
         
    if log:
        fig.update_layout(yaxis_type="log")
        title = title + " (log)"
    
    if rolling > 1:
        title = title + f" ({rolling} day rolling avg)"
        
        
    t_zero = pd.Timestamp(year = 2020, month = 1, day = 15,  
                  hour = 0, minute=0, second = 0)
    
    new_x_length = 0
    for country in filtered.Country.unique():
        df = filtered.loc[filtered['Country'] == country]
        days = df['Days'].head(1).values[0]
        
        # addresses case when a country didn't hit 50 threshold yet
        if not np.isnan(days):
            offset = pd.to_datetime("now") - t_zero - pd.Timedelta(days, unit='D')

            for cat in categories:
                if len(categories) > 1:
                    name=country+"_"+cat
                else:
                    name=country

                with pd.option_context('mode.chained_assignment', None):
                    df['Date'] = df['Date'] - offset
                    df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

                # let's figure out where t0 sits in the series are rework the x-axis based on it
                for i,d in enumerate(df['Date']):
                    if d == "2020-01-15":
                        t0 = i

                days_since_t0 = len(df['Date'])-t0        
                new_x = list(range(0-t0, days_since_t0, 1))
                
                # figure out which country has most x-values since t0 as this will be range for the x-axis
                if days_since_t0 > new_x_length:
                    new_x_length = days_since_t0

                # let's calculate rolling avg
                with pd.option_context('mode.chained_assignment', None):
                    df[cat] = df[cat].rolling(window=rolling).mean()

                fig.add_trace(go.Scatter(
                                x=new_x,
                                y=df[cat],
                                name=name))
    
    fig.update_layout(title_text=title, xaxis_range=[0, new_x_length-1])
    fig.show()

    
def print_pie(df, c_list, category):
    last_date = df.iloc[[-1]]['Date']
    df = df[df['Date']==last_date.values[0]]
    df=df.sort_values(by=['Country'])
    fig = go.Figure()
    fig.update_layout(title_text=category)
    fig.add_trace(go.Pie(labels=df['Country'], values=df[category]))
    fig.show()

def print_bar(df, c_list, category, log=True):
    if c_list[0] == 'All':
        filtered = df.copy()
    else:
        filtered = df[df['Country'].isin(c_list)]
 
    last_date = filtered.iloc[[-1]]['Date']
    filtered = filtered[filtered['Date']==last_date.values[0]]
    filtered=filtered.sort_values(by=['Country'])
    fig = go.Figure()    
    title = category
    
    if log:
        fig.update_layout(yaxis_type="log")
        title = title + " (log)"

    fig.update_layout(title_text=title)
    fig.add_trace(go.Bar(x=filtered['Country'], y=filtered[category]))
    fig.show()

def print_bars(df, c_list, categories, group_type='Country', title='Combined', days=5, log=False):
    if c_list[0] == 'All':
        filtered = df.copy()
        countries = df.Country.unique()
    else:
        filtered = df[df['Country'].isin(c_list)]
        countries = c_list        

    results = pd.DataFrame()
    for country in countries:  
        df = filtered.loc[filtered['Country'] == country]
        df = df.iloc[-days:]
        results = pd.concat([results, df])
        
    results['Date'] = results['Date'].dt.strftime('%m-%d')

    fig = go.Figure()
    if log:
        fig.update_layout(yaxis_type="log")
        title = title + " (log)"

    fig.update_layout(title_text=title, barmode="stack")
    for cat in categories:
        if group_type=='Country':
            fig.add_trace(go.Bar(x=[results['Country'],results['Date']],
                                    y=results[cat], name=cat ))
        if group_type=='Category':
            fig.add_trace(go.Bar(x=[[cat]*days,results['Date']],
                                    y=results[cat], name=cat))         
    fig.show()


def print_table(df, c_list):
    if c_list[0] == 'All':
        filtered = df.copy()
    else:
        filtered = df[df['Country'].isin(c_list)]
    
    last_date = filtered.iloc[[-1]]['Date']
    results = filtered[filtered['Date']==last_date.values[0]]
    results=results.sort_values(by=['Confirmed'], ascending=False)
    results=results[['Country', 'Date', 'Days', 'Confirmed', 'Confirmed_Delta', 'Confirmed_per_million', 'Confirmed_Pct', 'Deaths', 'Deaths_Delta']]
    display(results)   
    

## Full dataset

In [ ]:
display(agg)

# Graphs and tables

## Setup countries of interest and days to eval

In [ ]:
# c_lists are a series of lists with Countries of interest; change as appropriate
c_list1 = ['Netherlands', 'France', 'Germany', 'Belgium', 'Norway', 'Sweden', 'Switserland', 
          'Portugal', 'US', 'United Kingdom', 'Australia', 'Bosnia and Herzegovina',
          'Serbia', 'Italy', 'Spain', 'Korea, South']

c_list2 = ['Netherlands', 'France', 'Germany', 'Belgium', 'Sweden', 
          'Portugal', 'China', 'US', 'United Kingdom', 'Australia',
          'Brazil', 'Italy', 'Spain', 'Korea, South', 'Russia', 'Turkey']

c_list3 = ['Netherlands', 'France', 'Spain', 'Germany', 'Belgium', 'Italy', 'Sweden', 'UK', 'US']


# c_lists is a dict that is used to provide a drop-down option to select different countries list
# when displaying the figures
c_lists = {'c_list1 - Countries of interest' : c_list1,
           'c_list2 - w/ China, Russia' : c_list2,
           'c_list3 - ~W-Europe+US' : c_list3,
           'Netherlands' : ['Netherlands'],
           'All' : ['All']
          }

# Number of days to consider; efffectively sets the x-axis for the graphs
days=45
display(Markdown(f'#### Days: {days}'))


# Results

## Table - latest data all countries

In [ ]:
print_table(agg,['All'])

## Breakout of case-type per country

### Confirmed

In [ ]:
last_date = agg.iloc[[-1]]['Date']
df = agg[agg['Date']==last_date.values[0]]
display(Markdown(f"#### Total confirmed cases: {df['Confirmed'].sum()}"))

print_totals_graph(agg,'Confirmed_Delta', title='Confirmed Worldwide Daily Delta', rolling=5, log=False)
print_pie(agg, ['All'], 'Confirmed')
print_bar(agg, ['All'], 'Confirmed')


### Deaths

In [ ]:
last_date = agg.iloc[[-1]]['Date']
df = agg[agg['Date']==last_date.values[0]]
display(Markdown(f"#### Total death cases: {df['Deaths'].sum()}"))

print_totals_graph(agg,'Deaths_Delta', title='Deaths Worldwide Daily Delta', rolling=5, log=False)

print_pie(agg, ['All'], 'Deaths')
print_bar(agg, ['All'], 'Deaths')

## Confirmed cases

### Confirmed - Log-scale

In [ ]:
def update_confirmed_log_graph(selected):
    print_graph(agg, selected, ['Confirmed'], rolling=5, days=days)

widgets.interact(update_confirmed_log_graph, selected=widgets.Dropdown(options=c_lists, style={'description_width': 'initial'}, description='Select country'))


#print_graph(agg, c_list, ['Confirmed'], days=days)

### Confirmed - Linear scale

In [ ]:
def update_confirmed_graph(selected):
    print_graph(agg, selected, ['Confirmed'], days=days, rolling=5, log=False)

widgets.interact(update_confirmed_graph, selected=widgets.Dropdown(options=c_lists, style={'description_width': 'initial'}, description='Select country'))


### Confirmed - daily delta

In [ ]:
def update_confirmed_daily_delta(selected):
    print_bar(agg, selected, 'Confirmed_Delta', log=False)

widgets.interact(update_confirmed_daily_delta, selected=widgets.Dropdown(options=c_lists, style={'description_width': 'initial'}, description='Select country'))


### Confirmed - daily delta graph

In [ ]:
def update_confirmed_daily_delta_graph(selected):
    print_graph(agg, selected, ['Confirmed_Delta'], days=days, rolling=5, log=False, trendline=False)

widgets.interact(update_confirmed_daily_delta_graph, selected=widgets.Dropdown(options=c_lists, style={'description_width': 'initial'}, description='Select country'))



### Confirmed per million of population

In [ ]:
def update_confirmed_pm_graph(selected):
    print_graph(agg, selected, ['Confirmed_per_million'], rolling=5, days=days, log=False)

widgets.interact(update_confirmed_pm_graph, selected=widgets.Dropdown(options=c_lists, style={'description_width': 'initial'}, description='Select country'))


### 50 infections t0: confirmed cases

In [ ]:
def update_confirmed_t0_graph(selected):
    print_graph_from_start(agg, selected, ['Confirmed'], days=days, rolling=5, log=False)

widgets.interact(update_confirmed_t0_graph, selected=widgets.Dropdown(options=c_lists, style={'description_width': 'initial'}, description='Select country'))



### 50 infections t0: Confirmed per million

In [ ]:
def update_confirmed_t0_pm_graph(selected):
    print_graph_from_start(agg, selected, ['Confirmed_per_million'], rolling=5, days=days, log=False)

widgets.interact(update_confirmed_t0_pm_graph, selected=widgets.Dropdown(options=c_lists, style={'description_width': 'initial'}, description='Select country'))


### 50 infections t0: confirmed delta

In [ ]:
def update_confirmed_delta_t0_graph(selected):
    print_graph_from_start(agg, selected, ['Confirmed_Delta'], days=days, rolling=5, log=False)

widgets.interact(update_confirmed_delta_t0_graph, selected=widgets.Dropdown(options=c_lists, style={'description_width': 'initial'}, description='Select country'))


## Deaths

In [ ]:
def update_deaths_graph(selected):
    print_graph(agg, selected, ['Deaths'], days=days, rolling=5, log=False)


widgets.interact(update_deaths_graph, selected=widgets.Dropdown(options=c_lists, style={'description_width': 'initial'}, description='Select country'))



### Deaths - daily delta

In [ ]:
def update_deaths_daily_delta(selected):
    print_bar(agg, selected, 'Deaths_Delta', log=True)
    #print_bars(agg, selected, ['Deaths_Delta'], title='Last 3 days - Deltas', days=2, log=True)

widgets.interact(update_deaths_daily_delta, selected=widgets.Dropdown(options=c_lists, style={'description_width': 'initial'}, description='Select country'))


### Deaths - daily delta graph

In [ ]:
def update_deaths_daily_delta_graph(selected):
    print_graph(agg, selected, ['Deaths_Delta'], days=days, rolling=5, log=False, trendline=False)

widgets.interact(update_deaths_daily_delta_graph, selected=widgets.Dropdown(options=c_lists, style={'description_width': 'initial'}, description='Select country'))


### 50 infections t0: deaths

In [ ]:
def update_deaths_t0_graph(selected):
    print_graph_from_start(agg, selected, ['Deaths'], days=days, rolling=5, log=False)

widgets.interact(update_deaths_t0_graph, selected=widgets.Dropdown(options=c_lists, style={'description_width': 'initial'}, description='Select country'))

#print_graph_from_start(agg, c_list, ['Deaths'], days=days, log=False)
#print_graph_from_start(agg, ["All"], ['Deaths'], days=days, log=False)

### 50 infections t0: deaths per million

In [ ]:
def update_deaths_t0_pm_graph(selected):
    print_graph_from_start(agg, selected, ['Deaths_per_million'], rolling=5, days=days, log=False)

widgets.interact(update_deaths_t0_pm_graph, selected=widgets.Dropdown(options=c_lists, style={'description_width': 'initial'}, description='Select country'))

### 50 infections t0: deaths delta

In [ ]:
def update_confirmed_delta_t0_graph(selected):
    print_graph_from_start(agg, selected, ['Deaths_Delta'], days=days, rolling=5, log=False)

widgets.interact(update_confirmed_delta_t0_graph, selected=widgets.Dropdown(options=c_lists, style={'description_width': 'initial'}, description='Select country'))


### Combined

In [ ]:
c_list1=['Netherlands', 'Belgium', 'Italy', 'France', 'Spain', 'US', 'United Kingdom']
print_bars(agg, c_list1, ['Deaths', 'Confirmed'], title='Last 7 days - Totals', days=7, log=False)

### Combined delta

In [ ]:
c_list1=['Netherlands', 'Belgium', 'Italy', 'France', 'Spain', 'US', 'United Kingdom']
print_bars(agg, c_list1, ['Deaths_Delta', 'Confirmed_Delta'], title='Last 7 days - Deltas', days=7, log=False)

## Netherlands

### Combined case-types

In [ ]:
def update_combined_graph(selected):
    print_graph(agg, [selected], ['Confirmed', 'Deaths'], rolling=5, days=days, log=False)

widgets.interact(update_combined_graph, selected=widgets.Dropdown(options=c_list1, style={'description_width': 'initial'}, description='Select country'))


In [ ]:
def update_combined_bars(selected):
    print_bars(agg, 
           [selected], 
           ['Confirmed_Delta', 'Deaths_Delta'], 
           group_type='Category', days=60, log=False)
    
widgets.interact(update_combined_bars, selected=widgets.Dropdown(options=c_list1, style={'description_width': 'initial'}, description='Select country'))

